# Project 1: Machine Learning

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from implementations import *
import matplotlib as mpl
import matplotlib.pyplot as plt
from helper import load_csv_data
from processing import clean_data, standardize

# Import of the data
Import of the train and test data

In [3]:
y_train, x_train, id_train = load_csv_data("../data/train.csv")
print(x_train.shape, y_train.shape)

(250000, 30) (250000,)


In [ ]:
var_names = np.genfromtxt("../data/train.csv",
              delimiter=',',
              encoding='UTF-8-sig',
              dtype=None,
              names=True).dtype.names[2:]

In [ ]:
y_test, x_test, id_test = load_csv_data("../data/test.csv")
print(x_test.shape, y_test.shape)

(568238, 30) (568238,)


# Preprocess data
We pre process the data to get a clean dataset

In [5]:
x_train_cleaned, _, _ = standardize(clean_data(x_train))
x_train_cleaned = np.insert(x_train_cleaned, [0], np.ones((x_train.shape[0],1)), axis=1)

# Learning algorithms

### Least squares

In [6]:
weight, loss = least_squares(y_train, x_train_cleaned)
print(compute_mse_loss(y_train, x_train_cleaned, weight))

0.51004


### Least squares with ridges regression

In [7]:
weight, loss = ridge_regression(y_train, x_train_cleaned, 10)
print(loss)

0.509928


### Least squares with gradient descent

In [8]:
weight, loss = mean_squared_error_gd(y_train, x_train_cleaned, np.ones((31,)), 100, 1e-3)
loss

1.215536

### Least squares with stochastic gradient descent

In [9]:
weight, loss = mean_squared_error_sgd(y_train, x_train_cleaned, np.ones((31,)), 100, 1e-3)
loss

0.775736

### Logistic regression

In [10]:
w, l = logistic_regression(y_train, x_train_cleaned, np.ones((31,)), 1000, 1e-3)

Current iteration=0, loss=1.995336631273058
Current iteration=100, loss=1.0491363661898327
Current iteration=200, loss=0.27006069549426187
Current iteration=300, loss=-0.37199300383762574
Current iteration=400, loss=-0.8974960829542113
Current iteration=500, loss=-1.3222079469218488
Current iteration=600, loss=-1.661546133055785
Current iteration=700, loss=-1.9325063200387396
Current iteration=800, loss=-2.15229341612981
Current iteration=900, loss=-2.335662630699634


In [11]:
w, l = reg_logistic_regression(y_train, x_train_cleaned, 0.2,  np.ones((31,)), 1000, 1e-3)

Current iteration=0, loss=1.995336631273058
Current iteration=100, loss=0.9989272803604264
Current iteration=200, loss=0.20757768961068648
Current iteration=300, loss=-0.41169620130994156
Current iteration=400, loss=-0.8829184065559729
Current iteration=500, loss=-1.2271743752375226
Current iteration=600, loss=-1.4682340429038723
Current iteration=700, loss=-1.632840431601386
Current iteration=800, loss=-1.745383871074444
Current iteration=900, loss=-1.8236927657875832
